In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import rebound

%matplotlib inline

In [2]:
sim = rebound.Simulation()

In [3]:
sim.G = 1e-2
rho = 2000
radius= 1
mass = 4/3.*np.pi*radius**3
print(mass)

4.18879020479


In [4]:
sim.add(m=4., r=radius, x=-1, y=-1)
sim.add(m=4., r=radius, x=1, y=1)
sim.add(m=4., r=1, x=1.01, y=-1.01)

In [5]:
sim.integrator = 'leapfrog'
sim.move_to_com()
sim.collision = "none"

ps = sim.particles

In [6]:
def linear_spring_dashpot(r, v, Ri, Rj, kappa=300, gamma=0.5):    
    xi = max(0, Ri+Rj-r)
    return kappa*xi - gamma*np.sign(xi)*(-v)

In [7]:
def normal_force(reb_sim):
    for i in range(sim.N):
        for j in range(sim.N):
            if i == j:
                continue
            else:
                dx = ps[i].x - ps[j].x
                dy = ps[i].y - ps[j].y
                dz = ps[i].z - ps[j].z
                dist = math.sqrt(dx*dx + dy*dy + dz*dz)
    
                dvx = ps[i].vx - ps[j].vx
                dvy = ps[i].vy - ps[j].vy
                dvz = ps[i].vz - ps[j].vz
                vrel = math.sqrt(dvx*dvx + dvy*dvy + dvz*dvz)
                
                normforce = linear_spring_dashpot(dist, vrel, ps[i].r, ps[j].r)
                
                ps[i].ax += normforce*dx/dist
                ps[i].ay += normforce*dy/dist
                ps[i].az += normforce*dz/dist

In [8]:
sim.additional_forces = normal_force
sim.force_is_velocity_dependent = 1

In [9]:
# Set the output and how to save
Noutputs = 120
times = np.linspace(0.0, 120, Noutputs)
x = np.zeros((sim.N,Noutputs))
y = np.zeros((sim.N,Noutputs))

In [10]:
# integrate
for i, time in enumerate(times):
    sim.integrate(time)
    for ipar in range(sim.N):      
        x[ipar][i] = ps[ipar].x   # This stores the data which allows us to plot it later
        y[ipar][i] = ps[ipar].y

In [11]:
# Plot using matplotlib
for i in range(Noutputs):
    fig = plt.figure(figsize=(5, 5))
    ax = plt.subplot(111)
    ax.set_xlim(-3, 3)
    ax.set_ylim(-3, 3)
    
    for j in range(sim.N):
        circle = plt.Circle((x[j][i], y[j][i]), ps[j].r, lw=0, alpha=0.3)
        ax.add_artist(circle)
        
    plt.savefig("./image/tumbuk_"+str(i))
    plt.close()